# Clustering

In [35]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [36]:
import sys
import numpy as np
import scipy.sparse
# import sparsechem as sc
import argparse

In [37]:
for i in sys.path:
    print(i)

In [38]:
if ('../..' not in sys.path):
    sys.path.append('..')
print(sys.path)

In [39]:
import leader_follower as lf

In [40]:
def load_sparse(filename):
    """Loads sparse from Matrix market or Numpy .npy file."""
    if filename is None:
        return None
    if filename.endswith('.mtx'):
        return scipy.io.mmread(filename).tocsr()
    elif filename.endswith('.npy'):
        return np.load(filename, allow_pickle=True).item().tocsr()
    elif filename.endswith('.npz'):
        return scipy.sparse.load_npz(filename).tocsr()
    raise ValueError(f"Loading '{filename}' failed. It must have a suffix '.mtx', '.npy', '.npz'.")


In [41]:
def hierarchical_clustering(X, dists):
    """
    Args:
        X       compound matrix in CSR
        dists   list of (increasing) distances
    Sequentially clusters with each dists, returns final cluster ids
    """
    assert type(X) == scipy.sparse.csr.csr_matrix, "X should be csr_matrix (scipy.sparse)"
    
    print(f"X shape: {X.shape}")
    cl0 = np.arange(X.shape[0])
    Xc  = X

    for dist in dists:
        print(f"Running clustering for distance: {dist}")
        cl, cent = cluster(Xc, dist)
        Xc       = Xc[cent]
        cl0      = cl[cl0]

    return cl0


In [42]:
%load_ext Cython
# del cluster

In [66]:
%%cython --cplus --force

from libcpp.vector cimport vector
import numpy as np


def cluster(X, radius):

    print( f"X shape        :  {X.shape}")
    print( f"X.indptr shape :  {X.indptr.shape}")
    print( f"X.indices shape:  {X.indices.shape}")
    
    cdef int i, j, k, row, col, closest
    cdef int [:] x_indptr  = X.indptr
    cdef int [:] x_indices = X.indices
#     cdef double [:] x_data = X.data

    cdef int X_shape0 = X.shape[0]
    cdef int X_shape1 = X.shape[1]

    ## algorithm part

    cdef vector[double]      dist
    cdef vector[vector[int]] centers
    cdef vector[double]      Cnorm
    cdef vector[int]         center_ids

    ## Xnorm: no of non-zero columns in each row
    cdef long   [:] Xnorm = np.array((X != 0).sum(axis = 1)).flatten()
    cdef double [:] dists = np.zeros(X_shape0, dtype=np.float64)

    ## Clusters[] : cluster assignment of each X row
    cdef long [:] clusters = np.zeros(X.shape[0], dtype = np.int) - 1
    cdef long [:] perm_ids = np.random.permutation(X.shape[0])       

    cdef double min_dist, tmp_dist
    cdef int num_changed
    
#     print(f" ==== perm ids:       {np.asarray(perm_ids)[0:100]}")

    print(f" ==== Xnorm shape:    {Xnorm.shape}")
    print(f" ==== Xnorm size:     {Xnorm.shape}")
    print(f" ==== Xnorm:          {np.asarray(Xnorm)[:50]}" )
    print()

    print(f" ==== dists size:     {dists.shape}")
    print(f" ==== dists:          {np.asarray(dists)[:50]}")
    print()

    print(f" ==== dist size:      {dist.size()}")
    print(f" ==== dist :          {np.asarray(dist)[:50]}")    
    print()
    
    print(f" ==== centers size    {centers.size()}")
    centers.resize(X_shape1)
    print(f" ==== centers size    {centers.size()}")
    print()
    print(f" ==== Cnorm size      {Cnorm.size()}")  
    print(f" ==== Cnorm :         {np.asarray(Cnorm)[:50]}") 
    print()
    
    print(f" ==== center_ids size {center_ids.size()}")    
    print(f" ==== center_ids      {center_ids}")    
    print()    
    
    print(f" ==== clusters size:  {clusters.shape}")
    print(f" ==== clusters  :     {clusters.shape}")
    print()    
    
    print(f" ==== perm_ids size:  {perm_ids.shape}")
    print(f" ==== perm_ids :      {np.asarray(perm_ids)[:10]}. . . . .")
          
    for i in range(X_shape0):
        if i > 20000: 
            return
        if i % 10000 == 0:
            print(f"Row {i}.")
                
        row = perm_ids[i]
        print(f">> i: {i:2d} - row = perm_ids[i]: {perm_ids[i]:6d}  - x_indptr[{row:6d}]: "
              f"{x_indptr[row]:9d}  x_indptr[{row+1:6d}]: {x_indptr[row+1]:9d}  Cnorm: {Cnorm.size()}")
        
        ## computing distances to all centers
        dists[0 : Cnorm.size()] = 0.0
        
        ## for current row, get column indices and populated columns data for current row 
        
        for j in range(x_indptr[row], x_indptr[row+1]):
            col = x_indices[j]
            print(f"    2.1  row: {row} col= x_indices[{j}] = {col}   k=centers[{col}].size: {centers[col].size()}   {centers[col]}")
            for k in range(centers[col].size()):
                dists[ centers[col][k] ] += 1.0
                print(f"    2.2  set dists[centers[{col}][{k}]] : dists[{centers[col][k]}] += 1 -> {dists[ centers[col][k] ]}" )
                
                
        closest = -1
        min_dist = radius
        
        ## Calculate Distance from Centroids 
        for j in range(Cnorm.size()):
#             print(f"   3 - j: {j} of {Cnorm.size()} :  dists[j] = 1.0 - {dists[j]} / ({Xnorm[row]} + {Cnorm[j]} - {dists[j]})")            
            dists[j] = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
            if dists[j] < min_dist:
                min_dist = dists[j]
                closest  = j

        if closest >= 0:
            clusters[row] = closest
            continue

        ## create a new cluster
#         print(f"   4 - Create a new cluster k = Cnorm.size() = {Cnorm.size()} Xnorm[row] : {Xnorm[row]}")
        k = Cnorm.size()
        
        for j in range(x_indptr[row], x_indptr[row+1]):
            ## Add new element at end of centers
            centers[ x_indices[j] ].push_back(k)
#             print(f"       j: {j} - :  add k = Cnorm.size() = {k} to centers[{x_indices[j]}]: {centers[ x_indices[j] ].size()}")
                
        clusters[row] = k
        Cnorm.push_back(Xnorm[row])
        center_ids.push_back(row)
#         print(f"       clusters[{row}] = {k}")
#         print(f"       Cnorm.add(Xnorm[{row}] = {Xnorm[row]}):  new Cnorm size:{Cnorm.size()}")
#         print("      ", Cnorm)
#         print(f"       center_ids.add(row: {row}): new center_ids size:{center_ids.size()}")
#         print("      ", center_ids)
        
    print("==============================================")
    print("Reassigning compounds to the closest clusters.")
    print("==============================================") 
    num_changed = 0
    
    for row in range(X_shape0):
        if row % 50000 == 0:
            print(f"Row {row}.")
        ## compute distances to all clusters, assign to the closest
        for j in range(x_indptr[row], x_indptr[row+1]):
            col = x_indices[j]
            for k in range(centers[col].size()):
                dists[ centers[col][k] ] += 1.0

        closest = -1
        min_dist = radius + 1e-5
        
        # calculate distance to Clusters
        for j in range(Cnorm.size()):
            try:
                tmp_dist = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
            except ZeroDivisionError:
                print(' Zero Division Error Encountered  - row:', row)
                print(' Xnorm[row] ', Xnorm[row])
                print(' Cnorm[j]   ', Cnorm[j])
                print(' dists[j]   ', dists[j])
            else:
                if tmp_dist < min_dist:
                    min_dist = tmp_dist
                    closest  = j
                    if min_dist == 0:
                        ## best possible
                        break
        if (closest >= 0) and (clusters[row] != closest):
            clusters[row]  = closest
            num_changed   += 1

    print(f"Reassignement changed {num_changed} assignments.")
    print(f"Total {len(center_ids)} clusters.")

    return np.asarray(clusters), np.asarray(center_ids)

### main

In [59]:
parser = argparse.ArgumentParser(description="Training a multi-task model.")
parser.add_argument("--x"  , help="Descriptor file (matrix market or numpy)", type=str, required=True)
parser.add_argument("--out", help="Output file for the clusters (.npy)", type=str, required=True)
parser.add_argument("--dists", nargs="+", help="Distances", default=[], type=float, required=True)


input_args =  "python ../leader_follower/cluster.py --x output/20_chembl_29_X.npy --out output/30_clustering.npy --dist 0.5 0.7".split()
print(input_args)

args = parser.parse_args(input_args[2:])
print(vars(args))
dists = args.dists


In [60]:
print(f"Loading '{args.x}'.")
# X = sc.load_sparse(args.x).tocsr()
X = load_sparse(args.x).tocsr()


X.shape

# Xnorm = np.array((X != 0).sum(axis = 1)).flatten()

# Xnorm.shape


### Hierarchical Clustering

In [ ]:
print("Clustering.")
# two step clustering, first at 0.5, then at 0.6
cl_hier = hierarchical_clustering(X, dists=args.dists)

In [ ]:
np.save(args.out, cl_hier)
print(f"Saved clusters into '{args.out}'.")

In [ ]:
type(cl_hier)
cl_hier

In [50]:
print(f"Loading '{args.x}'.")
# X = sc.load_sparse(args.x).tocsr()
X_orig = load_sparse(args.x).tocsr()
X = X_orig[:100, :15000]
print(" X.indptr : " , len(X.indptr), X.indptr)
print(" X.indices: " , len(X.indices) , X.indices)
print(X)
X

In [ ]:
print(f"X shape: {X.shape}")
cl0 = np.arange(X.shape[0])
Xc  = X

In [ ]:
for dist in dists:
    print(f"Running clustering for distance: {dist}")
    cl, cent = cluster(Xc, dist)
    Xc       = Xc[cent]
    cl0      = cl[cl0]




### `clustering()` Line by line

In [ ]:
from libcpp.vector cimport vector
import numpy as np
import scipy.sparse

In [ ]:
X_shape0 = X.shape[0]
X_shape1 = X.shape[1]
x_indptr  = X.indptr
x_indices = X.indices

print(X_shape0, X_shape1, len(x_indptr), len(x_indices))

In [ ]:
## Number of non zero columns 
Xnorm = np.array((X != 0).sum(axis = 1)).flatten()

print(Xnorm.shape)
print(Xnorm.sum())

In [ ]:
dists = np.zeros(X.shape[0], dtype=np.float64)
print(dists.shape)

In [ ]:
clusters = np.zeros(X.shape[0], dtype = int) - 1
perm_ids = np.random.permutation(X.shape[0])       

print(clusters)
print(perm_ids)

In [ ]:

for i in range(X_shape0):
    if i % 10000 == 0:
        print(f"Row {i}.")
    row = perm_ids[i]

    ## computing distances to all centers
    dists[0 : Cnorm.size()] = 0.0
    for j in range(x_indptr[row], x_indptr[row+1]):
        col = x_indices[j]
        for k in range(centers[col].size()):
            dists[ centers[col][k] ] += 1.0

    closest = -1
    min_dist = radius
    for j in range(Cnorm.size()):
        dists[j] = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
        if dists[j] < min_dist:
            min_dist = dists[j]
            closest  = j

    if closest >= 0:
        clusters[row] = closest
        continue

    ## create a new cluster
    k = Cnorm.size()
    for j in range(x_indptr[row], x_indptr[row+1]):
        centers[ x_indices[j] ].push_back(k)
    clusters[row] = k
    Cnorm.push_back(Xnorm[row])
    center_ids.push_back(row)

print("Reassigning compounds to the closest clusters.")
num_changed = 0
for row in range(X_shape0):
    if row % 10000 == 0:
        print(f"Row {row}.")
    ## compute distances to all clusters, assign to the closest
    for j in range(x_indptr[row], x_indptr[row+1]):
        col = x_indices[j]
        for k in range(centers[col].size()):
            dists[ centers[col][k] ] += 1.0

    closest = -1
    min_dist = radius + 1e-5
    for j in range(Cnorm.size()):
        tmp_dist = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
        if tmp_dist < min_dist:
            min_dist = tmp_dist
            closest  = j
            if min_dist == 0:
                ## best possible
                break
    if (closest >= 0) and (clusters[row] != closest):
        clusters[row]  = closest
        num_changed   += 1

print(f"Reassignement changed {num_changed} assignments.")
print(f"Total {len(center_ids)} clusters.")

return np.asarray(clusters), np.asarray(center_ids)